<a href="https://colab.research.google.com/github/jespimentel/analisador_lotes_ip_colab/blob/main/litellm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANALISADOR DE IPS EM LOTE
### José Eduardo de Souza Pimentel

O programa emprega o poder do Python e das LLMs, mediadas pela biblioteca `litellm`, para analisar uma coleção de inquéritos policiais em formato PDF encontrados em determinada pasta (diretório).

### Estratégias
1. Usamos o `PyPDF` para extrair textos dos PDFs. Preservamos a numeração das páginas para referenciar as respostas.
2. O texto extraído compõe um extenso prompt, com as perguntas à LLM dirigidas à analise de cada caso.
3. A biblioteca `litellm` facilita o desenvolvimento e abstrai toda a complexidade das chamadas a APIs, fazendo fácil substituir um provedor de LLM por outro.

In [1]:
# Instalações

!pip install PyPDF2 litellm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.4 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [2]:
# Importações

import os
import requests
import PyPDF2
import re
from litellm import completion
from google.colab import userdata

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [3]:
# Configurações

#modelo = 'gpt-4o-mini'
#api_key = userdata.get('OPENAI_KEY')

modelo = 'gemini/gemini-1.5-flash'
api_key = userdata.get('GOOGLE_KEY')

#modelo = 'deepseek/deepseek-chat'
#api_key = userdata.get('DEEPSEEK_KEY')

diretorio_pdfs = "/content/" # Opcionalmente, escolha uma pasta do seu Google Drive

In [4]:
# Funções

def listar_pdfs(diretorio):
    pdfs = []
    for arquivo in os.listdir(diretorio):
        if arquivo.lower().endswith(".pdf"):
            pdfs.append(os.path.join(diretorio, arquivo))
    return pdfs

def ler_pdf(caminho_pdf):
    with open(caminho_pdf, 'rb') as arquivo:
        leitor = PyPDF2.PdfReader(arquivo)
        texto = ''
        for pagina_num, pagina in enumerate(leitor.pages, start=1):
            texto += f"--- Página {pagina_num} ---\n"
            texto += pagina.extract_text() + "\n\n"
    return texto

def limpar_texto(texto):
  # Remover a assinatura
  padrao_assinatura = r"Este documento .*?fls\. \d+"
  texto = re.sub(padrao_assinatura, "", texto, flags=re.DOTALL)
  # Remover sequências de números e barras
  padrao_sequencias = r"\/[j\d]+ (?:\/[j\d]+)+"
  texto_limpo = re.sub(padrao_sequencias, "", texto)
  return texto_limpo

def analisar_conteudo (texto, prompt, modelo, api_key):
  try:
    resultado = completion(
    model=modelo,
    messages=[{"role": "system", "content": "Você é um promotor de justiça especializado em analisar inquéritos policiais."},
                {"role": "system", "content": "Trabalhe somente com texto fornecido com o prompt. Não alucine."},
                {"role": "user", "content": f"**Texto do PDF:**\n\n{texto}"},
                {"role": "user", "content": f"**Instruções:**\n {prompt}"}],
                api_key=api_key,
                temperature=0.3,
                #max_tokens=5000,
    )
    return resultado.get('choices', [{}])[0].get('message', {}).get('content', 'Sem resposta.')
  except Exception as e:
    print(f"Erro: {e}")
    return None

def gerar_markdown(texto, nome_arquivo="analises.md"):
  modo = 'a' if os.path.exists(nome_arquivo) else 'w'
  try:
    with open(nome_arquivo, modo, encoding='utf-8') as arquivo:
      if modo == 'w':
        arquivo.write("# PROMOTORIA DE JUSTIÇA DE PIRACICABA\n\n")
        arquivo.write(texto)
      else:
         arquivo.write(texto)
  except Exception as e:
    print(f"Erro ao salvar o arquivo: {e}")

In [5]:
prompt = f"""
    Extraia as seguintes informações do texto fornecido.
    Inclua os números de páginas de onde as informações foram extraídas.
    Para compor as respostas, despreze as páginas sem informação ou incompreensíveis.
    Não invente nenhuma informação (não alucine).

    **Informações gerais**
    * Número do procedimento no padrão CNJ:
    * Data, hora e local da ocorrência:
    * Crime que está sendo apurado e dispositivo legal correspondente:
    * Indiciado(s) e sua(s) conduta(s):
    * Vítima(s) e testemunha(s) com seus respectivos relatos:

    **Laudo**
    * Resumo dos dados relevantes do laudo (se aplicável):
    * Tipos e quantidades de drogas apreendidas (se aplicável):

    **Avaliação da prova**
    * Classifique a prova como "boa" ou "fraca":
    * Justifique sua classificação:

    **Dúvidas e verificação**
    * Liste as dúvidas que você teve ao gerar o resumo:
    * Verifique se todos os dados fornecidos estão no PDF e confirme:

    **Resumo do caso**
    * Forneça um resumo, em 2 ou 3 parágrafos. Indique data, hora, local, indiciado(s), conduta(s) e toda informação juridicamente relevante:
    """

In [ ]:
pdfs = listar_pdfs(diretorio_pdfs)
conteudo = ""
for pdf in pdfs:
    texto = ler_pdf(pdf)
    texto_limpo = limpar_texto(texto)
    resultado = analisar_conteudo(texto_limpo, prompt, modelo, api_key)
    if resultado:
        conteudo += f"\n**Arquivo:** {pdf}\n\n"
        conteudo += resultado
        conteudo += "\n___________________________________\n"
        print(conteudo)
        gerar_markdown(conteudo)
    else:
        print(f"Não foi possível obter o resultado de {pdf}.")